In [1]:
import numpy as np
import itertools
from sklearn.externals import joblib
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
from functools import partial
from itertools import count
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
import string
import re
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import KFold

//anaconda/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 500 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = train.shape[1]
n_classes = 2 

NameError: name 'train_col' is not defined

In [3]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

NameError: name 'n_input' is not defined

In [ ]:
# Create model
def multilayer_perceptron(_X, _weights, _biases):
    #Hidden layer1 with RELU activation
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    #Hidden layer2 with RELU activation
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [ ]:
# Define loss and optimizer
# Softmax loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
# Adam Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
class DataSet(object):
    def __init__(self, features, labels):
        self._features = features
        self._labels = labels
        self._num_examples = labels.shape[0]
        self._epochs_completed = 0
        self._index_in_epoch = 0
    @property
    def features(self):
        return self._features
    @property
    def labels(self):
        return self._labels
    @property
    def num_examples(self):
        return self._num_examples
    @property
    def epochs_completed(self):
        return self._epochs_completed 
    def next_batch(self, batch_size):
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Shuffle the data
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._features = self._features[perm]
            self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size
            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._features[start:end].todense().astype(float), self._labels[start:end].astype(float)

In [ ]:
# Loading the data inti the class DataSets
data_sets = DataSets()
data_sets.train = DataSet(train_matrix, one_hot(train_labels, n_classes))
data_sets.validation = DataSet(val_matrix, one_hot(val_labels, n_classes))
data_sets.test = DataSet(test_matrix, one_hot(test_labels, n_classes))

In [ ]:
# Clean up the CSR to save memory
del train_matrix, val_matrix, train_labels, val_labels, test_matrix, test_labels

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(data_sets.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            # Load a new batch from train set
            batch_xs, batch_ys = data_sets.train.next_batch(batch_size)
            
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    # Get prediction probabilities 
    predict_probability = tf.nn.softmax(pred)[:, 1]
    
    # Evaluate with test data
    total_batch_tst = int(data_sets.test.num_examples/batch_size)
    #avg_accuracy = 0.
    
    predictproba = np.zeros(data_sets.test.num_examples)
    for i in range(total_batch_tst):
        # Load a new batch from test set
        batch_xs_tst, batch_ys_tst = data_sets.test.next_batch(batch_size)
        #avg_accuracy += accuracy.eval({x: batch_xs_tst, y: batch_ys_tst})/total_batch_tst
        #avg_accuracy += sess.run(accuracy, feed_dict={x: batch_xs_tst, y: batch_ys_tst})/total_batch_tst
        
        predictproba[i*batch_size : (i+1)*batch_size] = predict_probability.eval({x: batch_xs_tst, y: batch_ys_tst})
